In [40]:
# importing necessary modules
from pymongo import MongoClient as mc
#from pathlib import Path
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np
import pandas as pd
import fingerprint_enhancer
from walking import walking, walkonce, checkstable, mergeneighbors
import math

In [41]:
def get_image_paths_and_mv():
    # this funciton only return image paths from "FVC2000" folder
    main_lst = []
    mvs = []
    paths_list = []

    db_client = mc("10.5.18.101")
    coll = db_client.BI.FVC2000
    coll = coll.find({})
    
    for i in coll:
        #if(so_far < count):
        main_lst.append(['../../../19CS91R05/datasets/' + i['path'] , i['mv'] ])
        paths_list.append('../../../19CS91R05/datasets/' + i['path'])
        mvs.append(i['mv'])
    
    return main_lst

In [42]:
def walk_core2(en_image):
	detect_SP = walking(en_image)
	return detect_SP

In [43]:
# Lets check the Minutiae binary pattern

def checkIfPossible(x,y,image_height, image_width):
    if(x<0 or x>=image_height):
        return False
    elif(y<0 or y>=image_width):
        return False
    return True

def GetMinutiaBinaryPattern(minutiae, fingerprint):
    MBP_List = []
    # fingerprint = cv.imread(fingerprint_path, cv.IMREAD_GRAYSCALE)
    fingerprint  = fingerprint_enhancer.enhance_Fingerprint(fingerprint)
    image_height, image_width = fingerprint.shape

    for i in minutiae:
        x, y = i[:2]
        window = [[0 for i in range(3)] for j in range(3)]
        if(checkIfPossible(x-1, y+1,image_height,image_width)):
            window[0][0] = fingerprint[x-1][y+1]
        if(checkIfPossible(x, y+1,image_height,image_width)):
            window[0][1] = fingerprint[x][y+1]
        if(checkIfPossible(x+1, y+1,image_height,image_width)):
            window[0][2] = fingerprint[x+1][y+1]
        if(checkIfPossible(x-1, y,image_height,image_width)):
            window[1][0] = fingerprint[x-1][y]
        if(checkIfPossible(x, y,image_height,image_width)):
            window[1][1] = fingerprint[x][y]
        if(checkIfPossible(x+1, y,image_height,image_width)):
            window[1][2] = fingerprint[x+1][y]
        if(checkIfPossible(x-1, y-1,image_height,image_width)):
            window[2][0] = fingerprint[x-1][y-1]
        if(checkIfPossible(x, y-1,image_height,image_width)):
            window[2][1] = fingerprint[x][y-1]
        if(checkIfPossible(x+1, y-1,image_height,image_width)):
            window[2][2] = fingerprint[x+1][y-1]
        
        temp = []
        
        # direction of MBP vector (0,0) -> (0,1) -> (0,2) -> (1,2) -> (2,2) -> (2,1) -> (2,0) -> (1,0)
        # Hard coding

        temp.append(window[0][0]//255)
        temp.append(window[0][1]//255)
        temp.append(window[0][2]//255)
        temp.append(window[1][2]//255)
        temp.append(window[2][2]//255)
        temp.append(window[2][1]//255)
        temp.append(window[2][0]//255)
        temp.append(window[1][0]//255)
        
        
        # for j in range(3):
        #     for k in range(3):
        #         if(k == 1 and j ==1):
        #             continue
        #         else:
        #             temp.append(window[j][k])
        MBP_List.append(temp)
    return MBP_List

In [44]:
def create_data_frame(lst):
    fingerprint_path, minutia = lst
    # global main_df
    img = cv.imread(fingerprint_path,cv.IMREAD_GRAYSCALE)
    en_image = fingerprint_enhancer.enhance_Fingerprint(img)
    # en_image_bw = cv.cvtColor(en_image, cv.COLOR_GRAY2BGR)
    distance = []
    sectors = []
    try:
        SP = walk_core2(en_image)
    except:
        return
    
    if(min(SP['core'].shape) == 0):
        if min(SP['delta'].shape) == 0:
            # NO Singular Point
            return
        else:
            CorePoint = (int(SP['delta'][0,0]), int(SP['delta'][0,1]))
    else:
        CorePoint = (int(SP['core'][0,0]), int(SP['core'][0,1]))

    for i in minutia:

        dis = ((CorePoint[0] - i[0])**2 + (CorePoint[1] - i[1])**2 )**0.5

        distance.append(dis)

        rads = math.atan2((i[1]- CorePoint[1]), (i[0]- CorePoint[0]))
        
        #if CorePoint[0] == i[0]:
        #    rads = 90
        #else:
        #    rads = math.atan((CorePoint[1] - i[1])/(CorePoint[0] - i[0]))


        degs = math.degrees(rads)
        if degs < 0:
            degs = 360 + degs
        
        sectors.append(int(degs//5 + 1 ))
    
    mbps = GetMinutiaBinaryPattern(minutia, en_image)
    pths = [fingerprint_path for i in range(len(minutia))]
    
    data = {
        "Distance" : distance,
        "Sector" : sectors,
        "MBP" : mbps,
        "Image Path" : pths 
    }
    
    new_df = pd.DataFrame(data)
    
    return new_df



In [45]:
from multiprocessing import Pool
main_df = pd.DataFrame(columns= ['Distance', 'Sector', 'MBP', 'Image Path'])
lst = get_image_paths_and_mv()
with Pool(10) as P:
    ans = P.map(create_data_frame, lst)

In [46]:
con = pd.concat(ans, ignore_index=True)
con.to_csv('FVC_2000_index_space.csv')
con

,Distance,Sector,MBP,Image Path
0,94.244363,58,"[0, 0, 0, 1, 1, 1, 1, 1]",../../../19CS91R05/datasets/FVC2000/Dbs/Db1_a/...
1,93.984041,59,"[0, 0, 0, 0, 1, 1, 1, 0]",../../../19CS91R05/datasets/FVC2000/Dbs/Db1_a/...
2,138.538803,65,"[0, 0, 0, 0, 0, 1, 1, 1]",../../../19CS91R05/datasets/FVC2000/Dbs/Db1_a/...
3,84.202138,58,"[1, 1, 1, 1, 1, 1, 1, 1]",../../../19CS91R05/datasets/FVC2000/Dbs/Db1_a/...
4,140.356688,43,"[0, 0, 0, 0, 0, 0, 0, 0]",../../../19CS91R05/datasets/FVC2000/Dbs/Db1_a/...
...,...,...,...,...
212280,162.114157,27,"[0, 0, 0, 0, 0, 0, 0, 0]",../../../19CS91R05/datasets/FVC2000/Dbs/Db3_a/...
212281,157.613451,26,"[0, 0, 0, 0, 0, 0, 1, 0]",../../../19CS91R05/datasets/FVC2000/Dbs/Db3_a/...
212282,132.003788,21,"[0, 0, 0, 0, 0, 0, 0, 0]",../../../19CS91R05/datasets/FVC2000/Dbs/Db3_a/...
212283,150.442680,21,"[0, 0, 0, 0, 0, 0, 0, 0]",../../../19CS91R05/datasets/FVC2000/Dbs/Db3_a/...


In [47]:
file_minutia = get_image_paths_and_mv()
file_minutia = file_minutia[0]
df = create_data_frame(file_minutia)

sort_main_df = pd.read_csv('FVC_2000_index_space.csv')

list_of_dfs = []
list_of_rows = []
lambda_hamming = lambda mbp, realmbp : sum([1 if mbp[i]==realmbp[i] else 0 for i in range(8) ])


for idx,row in df.iterrows():
    dis = row['Distance']
    sec = row['Sector']
    MBP = row['MBP']
    def hamming_dis(mbp):
        count = 0
        for i in range(8):
            if(mbp[i] == MBP[i]):
                count += 1
        # return True if count > 3 else False
        return count
    list_of_rows.append([dis, sec, MBP])
    right_rows = sort_main_df[(sort_main_df['Distance'] == dis) ] 
    right_rows['NEW_MBP'] = right_rows['MBP'].apply(hamming_dis, 'columns')
    # right_rows = right_rows[right_rows['NEW_MBP'] > 3]
    # print(right_rows)
    list_of_dfs.append(right_rows)

/tmp/ipykernel_19677/298375784.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  right_rows['NEW_MBP'] = right_rows['MBP'].apply(hamming_dis, 'columns')
/tmp/ipykernel_19677/298375784.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  right_rows['NEW_MBP'] = right_rows['MBP'].apply(hamming_dis, 'columns')
/tmp/ipykernel_19677/298375784.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [56]:
final = pd.concat(list_of_dfs)

In [57]:
print(final)

nmbp = final['NEW_MBP']

        Unnamed: 0    Distance  Sector                       MBP  \
0                0   94.244363      58  [0, 0, 0, 1, 1, 1, 1, 1]   
6550          6550   94.244363      58  [1, 1, 1, 0, 0, 0, 0, 1]   
6839          6839   94.244363      69  [1, 0, 0, 1, 1, 1, 1, 1]   
14995        14995   94.244363      22  [0, 0, 0, 1, 1, 1, 1, 0]   
27292        27292   94.244363      22  [0, 0, 0, 0, 0, 0, 0, 0]   
...            ...         ...     ...                       ...   
155            155  178.630904      12  [1, 0, 0, 0, 0, 0, 0, 0]   
39974        39974  178.630904      12  [0, 0, 0, 0, 0, 0, 0, 0]   
152325      152325  178.630904      25  [0, 0, 0, 0, 0, 0, 0, 0]   
188789      188789  178.630904      30  [1, 1, 1, 1, 1, 1, 1, 1]   
193653      193653  178.630904      18  [0, 0, 0, 0, 0, 0, 0, 0]   

                                               Image Path NEW_MBP  
0       ../../../19CS91R05/datasets/FVC2000/Dbs/Db1_a/...       0  
6550    ../../../19CS91R05/datasets/FVC2000/Dbs

In [64]:
len(sort_main_df)

212285

In [58]:
paths = (final['Image Path'])
print(len(paths))
path_lst = list(paths)
print(len(set(path_lst)))

4635
2156


In [59]:
path_dict = dict()

for i in path_lst:
    if i not in path_dict.keys():
        path_dict[i] = 1
    else:
        path_dict[i] += 1


In [60]:
maxm = -1
for i in path_dict.keys():
    maxm = max(maxm, path_dict[i])
print(maxm)

132


In [61]:
max_lst = []
for i in path_dict.keys():
    if path_dict[i] == maxm:
        max_lst.append(i)

In [62]:
max_lst

['../../../19CS91R05/datasets/FVC2000/Dbs/Db1_a/79_6.tif']

In [63]:
file_minutia[0]

'../../../19CS91R05/datasets/FVC2000/Dbs/Db1_a/79_6.tif'